In [1]:
import psycopg

from pathlib import Path
from psycopg.rows import dict_row
from fishsense_api_workflow_worker.database import Database
from fishsense_api_workflow_worker.models.camera import Camera
from fishsense_api_workflow_worker.models.dive import Dive
from fishsense_api_workflow_worker.models.image import Image
from fishsense_api_workflow_worker.models.laser_label import LaserLabel
from fishsense_api_workflow_worker.models.head_tail_label import HeadTailLabel
from tqdm import tqdm

/home/chris/Repos/school/e4e/fishsense/fishsense-api-workflow-worker/.venv/lib/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "HeadTailLabel" shadows an attribute in parent "SQLModel"
  warnings.warn(
/home/chris/Repos/school/e4e/fishsense/fishsense-api-workflow-worker/.venv/lib/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "LaserLabel" shadows an attribute in parent "SQLModel"
  warnings.warn(


In [3]:
FABRICANT_PASSWORD_PATH=Path("../.secrets/postgres_fabricant_password.txt")

with FABRICANT_PASSWORD_PATH.open() as f:
    fabricant_password = f.read().strip()

In [4]:
PG_CONN_STR = (
    f"postgres://postgres:{fabricant_password}@"
    f"fabricant-prod.ucsd.edu:5432/"
    f"postgres"
)

In [4]:
SELECT_CAMERAS_QUERY = "SELECT serial_number, name FROM cameras"

In [5]:
database = Database()
await database.init_database()

In [6]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_CAMERAS_QUERY)

    for row in rows:
        existing_camera = await database.select_camera_by_serial_number(row["serial_number"])
        if existing_camera is not None:
            print(existing_camera)
            continue

        camera = Camera(serial_number=row["serial_number"], name=row["name"])
        await database.insert_or_update_camera(camera)

serial_number='BJ6C69506' name='FSL-01' id=1
serial_number='BJ6C83748' name='FSL-02' id=2
serial_number='BJ6C85528' name='FSL-03' id=3
serial_number='BJ6C85524' name='FSL-04' id=4
serial_number='BJ6C67988' name='FSL-05' id=5
serial_number='BJ6C67987' name='FSL-06' id=6
serial_number='' name='FSL-08' id=7
serial_number='BJPA75620' name='FSL-09' id=8
serial_number='BJPA75624' name='FSL-11' id=9
serial_number='BJ6C67989' name='FSL-07' id=10
serial_number='BJPA75627' name='FSL-10' id=11


In [7]:
SELECT_DIVES_QUERY = "SELECT path, date FROM dives ORDER BY date"
SELECT_IMAGES_QUERY = "SELECT path, dive, date, image_md5, camera_sn FROM images ORDER BY date"

In [8]:
dives = []
images = []

with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    dive_rows = cur.execute(query=SELECT_DIVES_QUERY)
    for dive_row in dive_rows:
        if ".Trashes" in dive_row["path"]:
            continue

        dives.append(dive_row)

    image_rows = cur.execute(query=SELECT_IMAGES_QUERY)
    for image_row in image_rows:
        images.append(image_row)

In [9]:
for dive in tqdm(dives):
    existing_dive = await database.select_dive_by_path(dive["path"])
    if existing_dive is not None:
        continue

    dive_images = [i for i in images if i["dive"] == dive["path"]]
    image_dates = [i["date"] for i in dive_images]

    image_dates.sort()
    end_date = image_dates[-1]

    camera = await database.select_camera_by_serial_number(images[-1]["camera_sn"])

    dive = Dive(path=dive["path"], dive_datetime=end_date, camera_id=camera.id)
    await database.insert_or_update_dive(dive)

100%|██████████| 479/479 [00:10<00:00, 47.22it/s]


In [ ]:
dives = await database.select_dives()

for dive in tqdm(dives):
    images_per_dive = [i for i in images if i["dive"] == dive.path]
    images_per_dive.sort(key=lambda i: i["date"])

    for image_row in images_per_dive:
        existing_image = await database.select_image_by_path(image_row["path"])

        if existing_image is not None:
            continue

        existing_checksum = await database.select_image_by_checksum(image_row["image_md5"])

        camera = await database.select_camera_by_serial_number(image_row["camera_sn"])
        image = Image(path=image_row["path"], taken_datetime=image_row["date"], checksum=image_row["image_md5"], dive_id=dive.id, camera_id=camera.id, is_canonical=existing_checksum is None)
        await database.insert_or_update_image(image)

 19%|█▊        | 89/479 [03:29<2:27:18, 22.66s/it]

In [5]:
SELECT_LASER_LABELS_QUERY = "SELECT cksum, task_id, x, y FROM laser_labels"

In [ ]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_LASER_LABELS_QUERY)

    for row in rows:
        existing_label = await database.select_laser_label_by_task_id(row["task_id"])
        if existing_label is not None:
            print(existing_label)
            continue

        image = await database.select_image_by_checksum(row["cksum"])

        label = LaserLabel(label_studio_task_id=row["task_id"], x=row["x"], y=row["y"], image_id=image.id)
        await database.insert_or_update_laser_label(label)

In [ ]:
SELECT_CANONICAL_DIVES_QUERY = "SELECT path, priority FROM canonical_dives"

In [ ]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_CANONICAL_DIVES_QUERY)

    for row in rows:
        dive = await database.select_dive_by_path(row["path"])
        if dive is None:
            print(f"Could not find dive for path {row['path']}")
            continue

        dive.priority = row["priority"]

        await database.insert_or_update_dive(dive)

Could not find dive for path drive-download-20240307T1050Z/.Trashes/501
Could not find dive for path drive-download-20240307T1050Z/.Trashes/501/082923_Pool Calibration/082323_Slate_FSL01
Could not find dive for path drive-download-20240307T1050Z/.Trashes/501/082923_Pool Calibration/082323_Slate_FSL04
Could not find dive for path drive-download-20240307T1050Z/.Trashes/501/082923_Pool Calibration/082329_Slate_FSL05


In [ ]:
SELECT_HEAD_TAIL_LABELS_QUERY = "SELECT cksum, task_id, head_x, head_y, tail_x, tail_y FROM headtail_labels"

In [ ]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_HEAD_TAIL_LABELS_QUERY)

    for row in rows:
        existing_label = await database.select_head_tail_labels_by_task_id(row["task_id"])
        if existing_label is not None:
            print(existing_label)
            continue

        image = await database.select_image_by_checksum(row["cksum"])

        label = HeadTailLabel(label_studio_task_id=row["task_id"], head_x=row["head_x"], head_y=row["head_y"], tail_x=row["tail_x"], tail_y=row["tail_y"], image_id=image.id)
        await database.insert_or_update_head_tail_label(label)